### En este documento se unen todos los CSVs individuales mensuales en uno único.

Primero se improtan todas las librerías necesarias.

In [ ]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import json

El siguiente paso es capturar todos los CSVs mensuales.

In [ ]:
# Ruta a tus archivos CSV históricos
csv_files = glob.glob('../data/monthly data/*.csv')

# Verificar que se encontraron todos archivos CSV
print(csv_files)
print("CSV files found:", len(csv_files))

Una vez se hayan identificado todos los CSVs a unir, definimos todas las columnas que tienen en común, es decir, nombramos las columnas que tendrá el CSV unificado.

In [ ]:
common_cols = ['id', 'fecha', 'tipo_elem', 'intensidad', 'ocupacion', 'carga', 'vmed', 'error', 'periodo_integracion']

Vamos procesando en grupos de 10k filas por cada CSV y guardamos los datos en un nuevo CSV llamado `2018-2025 traffic.csv`.

In [ ]:
first_chunk = True

for file in csv_files:
    for chunk in pd.read_csv(file, sep=';', chunksize=1000000): # Miramos en chunks de 1 000 k filas para no cargar todo en memoria
    
        # Filtrar solo columnas que nos interesan y que existan en este archivo
        chunk = chunk[[col for col in common_cols if col in chunk.columns]]

        # Agregar al dataframe principal
        chunk.to_csv('../data/2018-2025 traffic.csv', mode='a', header=first_chunk, index=False)
        first_chunk = False
    print(f"{file} — procesado.")

Verificamos que se haya traspasado la información correctamente en nuestro nuevo archivo.

In [ ]:
df = pd.read_csv('../data/2018-2025 traffic.csv', sep=',', chunksize=1000000)
chunk = next(df)
chunk.head(5)

El último paso para finalizar la unificación de datos es poder ubicar cada elemento medidor de tráfico. No nos sirve de nada tener un ID de un medidor de intensidad del tráfico si no sabemos donde se ubica. Para hacerlo, primero realizamos un conteo de todos los IDs en nuestra base de datos. 

In [ ]:
conteo = {}

# Leer por chunks
for chunk in df:
    for valor in chunk['id'].dropna():
        if valor in conteo:
            conteo[valor] += 1
        else:
            conteo[valor] = 1

# Mostrar resultados
for tipo, cantidad in conteo.items():
    print(f"{tipo}: {cantidad}")

Cargamos la lista de IDs y la comparamos con una lista XML que incluye los mismos IDs y una ubicación asignada. Esta comparación implica que si se encuentra el ID en el archivo XML, también tenemos una ubicación exacta del elemento. Para optimizar el proceso, se guardará la ubicación en una lista de diccionarios, donde cada diccionario es un ID con su ubicación en coordenadas.

In [ ]:
# Cargar los IDs
lista_ids = list(conteo.keys())

# Parsear XML
tree = ET.parse('../data/pm.xml')
root = tree.getroot()

# Buscar coincidencias
resultado = []
for pm in root.findall('pm'):
    idelem = int(pm.find('idelem').text)
    if idelem in lista_ids:
        st_x = pm.find('st_x').text.replace(',', '.')
        st_y = pm.find('st_y').text.replace(',', '.')
        resultado.append({
            'id': idelem,
            'coordenada_x': float(st_x),
            'coordenada_y': float(st_y)
        })

Verificamos que se obtiene un total de 5306 IDs con sus correspondientes ubicaciones. Solo estos IDs nos resultarán útiles en nuestro análsis.

In [ ]:
print("Resultado:", resultado)
print("Cantidad de resultados:", len(resultado))

Para no perder esta información valiosa, se almacena en un txt donde cada fila es un ID con su ubicación correspondiente.

In [ ]:
with open('../data/coordenadas_ids.txt', 'w', encoding='utf-8') as f:
    for item in resultado:
        f.write(json.dumps(item) + '\n')